## 1. 马尔可夫奖励过程

### 1.1 MRP代码实现

In [23]:
def calculate_next_state_value(discount_factor, rewards, old_state_values, transition_probabilities):
    # 创建一个空列表new_state_values，用于存储计算得到的下一阶段的状态值函数。
    new_state_values = []
    # 使用for循环遍历old_state_values列表的索引，current_state变量表示当前状态的索引。
    for current_state in range(len(old_state_values)):
        # 检查当前状态是否为最后一个状态，如果不是最后一个状态，执行以下代码块。
        #print("current_state:",current_state, ". old_state_values:",old_state_values)
        if current_state != len(old_state_values) - 1:
            # 创建一个变量next_state_value_sum，用于累加与当前状态相连的后续状态值函数的加权和。
            next_state_value_sum = 0.0
            # 使用for循环遍历transition_probabilities[current_state]列表中的每个元素，transition变量表示状态转移概率的一个元素。
            for transition in transition_probabilities[current_state]:
                # 从transition元素中获取转移概率和下一状态的索引，并将其加权乘以下一状态的旧值函数，然后累加到next_state_value_sum上。
                transition_probability = transition[0]
                next_state_index = transition[1]
                next_state_value_sum += transition_probability * old_state_values[next_state_index]
                #print("transition_probability:",transition_probability," next_state_index:",next_state_index, " next_state_value_sum:",next_state_value_sum," old_state_values[next_state_index]:",old_state_values[next_state_index])

            # 根据当前状态的即时奖励和折扣因子，计算当前状态在下一阶段的预期收益，并将其添加到new_state_values列表中。
            new_state_value = rewards[current_state] + discount_factor * next_state_value_sum
            new_state_values.append(new_state_value)
            #print("new_state_value:",new_state_value," new_state_values:",new_state_values," rewards[current_state]:",rewards[current_state]," discount_factor:",discount_factor, " next_state_value_sum:",next_state_value_sum)
            #print("\n")
    #print("------------------------")

    # 在new_state_values列表末尾添加一个值为0的元素，表示Sleep状态的值函数为0。
    new_state_values.append(0.0)
    return new_state_values

In [24]:
# 定义γ
discount_factor = 1

# 报酬顺序：Class 1、Class 2、Class 3、Pass、Pub、Facebook、Sleep
rewards = [-2., -2., -2., 10., 1., -1., 0.]

# 全0初始化值函数
old_state_values = [0, 0, 0, 0, 0, 0, 0]

'''
定义了一个嵌套列表transition_probabilities，表示状态之间的转移概率
每个子列表表示当前状态的后续状态以及对应的转移概率。
transition_probabilities[0]表示Class 1的状态后续状态的转移概率，
其中[0.5, 1]表示以0.5的概率转移到Class 2，[0.5, 5]表示以0.5的概率转移到Facebook。
'''
transition_probabilities = [
    [[0.5, 1], [0.5, 5]],
    [[0.8, 2], [0.2, 6]],
    [[0.6, 3], [0.4, 4]],
    [[1, 6]],
    [[0.2, 0], [0.4, 1], [0.4, 2]],
    [[0.1, 0], [0.9, 5]],
    [[0, 0]]
]
# new_state_values用于存储每次迭代得到的新的状态值函数。
new_state_values = []

In [25]:
for i in range(100):
    new_state_values = calculate_next_state_value(discount_factor, rewards, old_state_values, transition_probabilities)
    # 使用新生成的值函数列表替换旧的值函数列表
    old_state_values = new_state_values

In [4]:
print(new_state_values)

[-12.418287702397645, 1.4703089405374907, 4.3371337110593915, 10.0, 0.8410368812854547, -22.318009521720207, 0.0]


### 1.2 MRP矩阵形式求解

In [26]:
import numpy as np

# 折扣因子γ
discount_factor = 1

# 状态转移概率矩阵
transition_matrix = np.array([[0, 0.5, 0, 0, 0, 0.5, 0],
                             [0, 0, 0.8, 0, 0, 0, 0.2],
                             [0, 0, 0, 0.6, 0.4, 0, 0],
                             [0, 0, 0, 0, 0, 0, 1],
                             [0.2, 0.4, 0.4, 0, 0, 0, 0],
                             [0.1, 0, 0, 0, 0, 0.9, 0],
                             [0, 0, 0, 0, 0, 0, 0]])

# 即时奖励矩阵
rewards = np.array([[-2], [-2], [-2], [10], [1], [-1], [0]])

# 构造单位矩阵
identity_matrix = np.eye(7)

# 直接使用np.linalg.inv()函数求解逆矩阵，并使用@操作符进行矩阵乘法运算
value_matrix = np.linalg.inv(identity_matrix - discount_factor * transition_matrix) @ rewards

# 打印输出计算得到的状态值函数矩阵
print(value_matrix)

[[-12.54320988]
 [  1.45679012]
 [  4.32098765]
 [ 10.        ]
 [  0.80246914]
 [-22.54320988]
 [  0.        ]]


## 2. 马尔可夫决策过程

### 2.1 MDP代码实现

In [27]:
def calculate_next_state_value(probability, rewards, old_values, transition_probabilities):
    # 创建一个空列表new_values，用于存储计算得到的下一阶段的状态值函数。
    new_values = []
    for state_index in range(len(old_values)):
        if state_index != len(old_values) - 1:
            state_sum = 0.0
            for transition_index in range(len(transition_probabilities[state_index])):
                if isinstance(transition_probabilities[state_index][transition_index][0], list):
                    # 如果转移是一个列表，则需要考虑多个子转移的情况
                    transition_sum = 0.0
                    for sub_transition in transition_probabilities[state_index][transition_index][0]:
                        transition_sum += old_values[sub_transition[0]] * sub_transition[1]
                    state_sum += probability * (rewards[transition_probabilities[state_index][transition_index][1]]
                                                + transition_sum)
                else:
                    # 单个转移的情况
                    state_sum += probability * (rewards[transition_probabilities[state_index][transition_index][1]]
                                                + old_values[transition_probabilities[state_index][transition_index][0]])
            new_values.append(state_sum)

    new_values.append(0.0)
    return new_values

In [28]:
# 策略参数，控制状态转移概率的权重
probability_pi = 0.5
# 定义即时奖励列表：study pass pub facebook quit sleep
rewards_list = [-2., 10., 1., -1., 0., 0.]
# 定义旧状态值列表
old_values_list = [0, 0, 0, 0, 0]
# 定义转移概率列表
transition_probabilities = [[[1, 0], [3, 3]],
                            [[2, 0], [4, 5]],
                            [[[[0, 0.2], [1, 0.4], [2, 0.4]], 2], [4, 1]],
                            [[0, 4], [3, 3]],
                            []]
# new_values_list用于存储迭代计算得到的新状态值。
new_values_list = []

In [29]:
for i in range(100):
    new_values_list = calculate_next_state_value(probability_pi, rewards_list, old_values_list, transition_probabilities)
    old_values_list = new_values_list

In [30]:
print(new_values_list)

[-1.3076923099959044, 2.6923076920317515, 7.384615384159404, -2.3076923112229597, 0.0]


### 2.2 MDP矩阵形式求解

In [31]:
def calculate_state_value(pi, lambd, transition_probabilities, rewards, identity_matrix):
    p_mat = np.matrix(transition_probabilities)
    r_mat = np.matrix(rewards)
    i_mat = np.matrix(identity_matrix)
    # 根据Bellman方程计算状态值
    v_mat = (i_mat - lambd * p_mat).I * r_mat
    return v_mat

In [32]:
pi = 0.5
lambd = 1

transition_probabilities = [[0, pi, 0, pi, 0],
                            [0, 0, pi, 0, pi],
                            [pi*0.2, pi*0.4, pi*0.4, 0, pi],
                            [pi, 0, 0, pi, 0],
                            [0, 0, 0, 0, 0]]
rewards = [[pi*-2 + pi*-1],
           [pi*-2 + pi*0],
           [pi*1 + pi*10],
           [pi*0 + pi*-1],
           [0]]

identity_matrix = np.eye(5)

state_values = calculate_state_value(pi, lambd, transition_probabilities, rewards, identity_matrix)

print(state_values)

[[-1.30769231]
 [ 2.69230769]
 [ 7.38461538]
 [-2.30769231]
 [ 0.        ]]


## 3. 贝尔曼最优方程

In [33]:
def calculate_next_state_value(policy_param, rewards_list, old_values_list, weight_list):
    new_values_list = []
    for state_index in range(len(old_values_list)):
        if state_index != len(old_values_list) - 1:
            max_list = []
            for transition_index in range(len(weight_list[state_index])):
                if isinstance(weight_list[state_index][transition_index][0], list):
                    transition_sum = 0.0
                    for sub_transition in weight_list[state_index][transition_index][0]:
                        transition_sum += old_values_list[sub_transition[0]] * sub_transition[1]
                    max_list.append(policy_param * (rewards_list[weight_list[state_index][transition_index][1]] + transition_sum))
                else:
                    max_list.append(policy_param * (rewards_list[weight_list[state_index][transition_index][1]]
                                                    + old_values_list[weight_list[state_index][transition_index][0]]))
            new_values_list.append(max(max_list))

    new_values_list.append(0.0)
    return new_values_list

In [34]:
policy_param = 1  # 策略参数，控制状态转移概率的权重
# 即时奖励列表，表示每个状态的即时奖励：study pass pub facebook quit sleep
rewards_list = [-2., 10., 1., -1., 0., 0.]
old_values_list = [0, 0, 0, 0, 0]  # 上一轮迭代的状态值列表
weight_list = [[[1, 0], [3, 3]],
               [[2, 0], [4, 5]],
               [[[[0, 0.2], [1, 0.4], [2, 0.4]], 2], [4, 1]],
               [[0, 4], [3, 3]],
               []]  # 权重列表，描述状态转移和即时奖励的对应关系

new_values_list = []
for i in range(100):
    new_values_list = calculate_next_state_value(policy_param, rewards_list, old_values_list, weight_list)
    old_values_list = new_values_list

print(new_values_list)

[6.0, 8.0, 10.0, 6.0, 0.0]
